In [37]:
import os
import tensorflow as tf
from dotenv import load_dotenv

In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\ML-OPs\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS\\Desktop\\ML-OPs\\Kidney-Disease-Classification'

In [38]:
load_dotenv()

True

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = os.getenv("MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [14]:
model = tf.keras.models.load_model("./artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [33]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="./artifacts/training/model.h5",
            training_data="./artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/keshav1017/Kidney-Disease-Classification.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )

        return eval_config

In [30]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 225, 
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluate(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("./results/scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # model registry does not work with file store
            if tracking_url_type_store != "file":
                # register the model
                mlflow.keras.log_model(self.model, artifact_path="model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model)

In [35]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[ 2025-06-27 16:44:04,685 ] INFO - common - yaml file: config\config.yaml loaded successfully.
[ 2025-06-27 16:44:04,700 ] INFO - common - yaml file: params.yaml loaded successfully.
[ 2025-06-27 16:44:04,705 ] INFO - common - Created directory at: artifacts


Found 93 images belonging to 2 classes.
6/6 [==============================] - 21s 3s/step - loss: 3.1566 - accuracy: 0.6237


[ 2025-06-27 16:44:26,722 ] INFO - common - json file saved at results\scores.json
[ 2025-06-27 16:44:27,205 ] DEBUG - connectionpool - https://dagshub.com:443 "POST /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow/runs/create HTTP/1.1" 200 None
[ 2025-06-27 16:44:27,650 ] DEBUG - connectionpool - https://dagshub.com:443 "POST /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 None
[ 2025-06-27 16:44:28,106 ] DEBUG - connectionpool - https://dagshub.com:443 "POST /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 None
2025/06/27 16:44:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
[ 2025-06-27 16:44:31,341 ] WARNING - save - Found untraced functions such as _jit_compiled_convolution_op, _jit

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpje8_ylrj\model\data\model\assets


[ 2025-06-27 16:44:32,786 ] INFO - builder_impl - Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpje8_ylrj\model\data\model\assets
[ 2025-06-27 16:44:48,982 ] DEBUG - connectionpool - https://dagshub.com:443 "GET /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow/runs/get?run_uuid=80509f7bc6244fde9c311102bb469fa9&run_id=80509f7bc6244fde9c311102bb469fa9 HTTP/1.1" 200 None
[ 2025-06-27 16:44:49,601 ] DEBUG - connectionpool - https://dagshub.com:443 "PUT /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/0802a22972a14fbc92f13bcf6945481f/80509f7bc6244fde9c311102bb469fa9/artifacts/model/conda.yaml HTTP/1.1" 200 None
[ 2025-06-27 16:44:50,235 ] DEBUG - connectionpool - https://dagshub.com:443 "PUT /keshav1017/Kidney-Disease-Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/0802a22972a14fbc92f13bcf6945481f/80509f7bc6244fde9c311102bb469fa9/artifacts/model/MLmodel HTTP/1.1" 200 None
[ 2025-06-27 16:44:50,829 ] DEBUG - connectionpool - 